In [ ]:
from nbpypath import *

In [ ]:
import numpy as np
import pandas as pd

from numba import njit


In [ ]:
from oscb import *

# customize Bokeh plotting in the Jupyter Notebook environment
setup_notebook()

In [ ]:
# use NLTK to get corpus data

import nltk

nltk.download('brown')

from nltk.corpus import brown


In [ ]:
# Seed the PRNG
random_seed(12345)

In [ ]:
lnet = LetterNet(
    MAX_SYNAPSES=1_000_000,  # max number of synapses, one million by default
    N_COLS_PER_LETTER=10,  # distributedness of letter SDR
    SPARSE_FACTOR=1,  # sparseness of letter SDR
    N_CELLS_PER_COL=100,  # per mini-column capacity
)

In [ ]:
%%time

lnet.learn_words_as_sequence(
    [], # no input data to see JIT time cost
    sp_width=(1, 1),  # width of spike train: [n_columns, n_cells]
    sp_thick=1,  # thickness of spike train
)


In [ ]:
%%time

lnet.learn_words(
    brown.words(
        # select part of the corpus if desirable, or
        # comment following line out and the full corpus is used
        categories=[ 
            'news', 
            'reviews', 
        ],
    ),
    sp_width=(1, 10),  # width of spike train: [n_columns, n_cells]
    sp_thick=10,  # thickness of spike train
    
    link_back=False,  # create backward synapses too
)


In [ ]:
# get a DataFrame about all excitatory synapses so far
excit_df = lnet.excitatory_synapses()
excit_df

In [ ]:
# statistics of individual synapses
excit_df.efficacy.describe()

In [ ]:
# statistics of all incoming synapses per individual postsynaptic neurons
excit_df.groupby(['to_column', 'to_cell'])['efficacy'].sum().describe()

In [ ]:
lsim = LetterNetSim(
    lnet,

    # global scaling factor, to accommodate a unit synaptic efficacy value of 1.0
    # roughly this specifies that:
    #   how many presynaptic spikes is enough to trigger a postsynaptic spike,
    #   when each synapse has a unit efficacy value of 1.0
    SYNAP_FACTOR=20,

    # reset voltage, equal to VOLT_REST, or lower to enable refractory period
    VOLT_RESET=-0.1,
    # membrane time constant
    τ_m=10,

    # fire plot params
    plot_n_steps=600,

    fire_dots_alpha=0.005,
)

for lcode in range(26):
    lsim.simulate(
        20, # n steps

        chr(ord("a") + lcode),
        prompt_blur=0,  # disable other signals
        force_prompt=True,  # force fire all cells as prompted
    )

show(lsim.fig)